In [27]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import gaussian_kde
from scipy.optimize import minimize

In [30]:
def optimize_portfolio_kde_cvar(tickers, start_date, end_date,  risk_profile='medio'):
    """
    Optimiza un portafolio según el perfil de riesgo.
    
    Args:
    - tickers: Lista de símbolos de las acciones.
    - start_date: Fecha de inicio para descargar datos.
    - end_date: Fecha de fin para descargar datos.
    - risk_profile: Perfil de riesgo ('bajo', 'medio', 'alto').
    
    Returns:
    - optimal_weights: Pesos óptimos para cada acción.
    - portfolio_return: Retorno esperado del portafolio.
    - portfolio_var: VaR del portafolio.
    """
    # Ajustar el nivel de confianza según el perfil de riesgo
    if risk_profile == 'bajo':
        confidence_level = 0.95
    elif risk_profile == 'medio':
        confidence_level = 0.90
    elif risk_profile == 'alto':
        confidence_level = 0.85
    else:
        confidence_level = 0.95  # valor por defecto

    # Descargar datos de Yahoo Finance
    data = yf.download(tickers, start=start_date, end=end_date)['Close']
    returns = np.log(data / data.shift(1)).dropna()
    n_assets = len(tickers)

    def calculate_cvar(weights):
        portfolio_returns = np.dot(returns, weights)
        kde = gaussian_kde(portfolio_returns)
        x = np.linspace(portfolio_returns.min(), portfolio_returns.max(), 1000)
        kde_values = kde(x)
        cumulative_values = np.cumsum(kde_values) / np.sum(kde_values)
        var_index = np.where(cumulative_values >= (1 - confidence_level))[0][0]
        var_value = x[var_index]
        cvar_estimation = np.mean(portfolio_returns[portfolio_returns <= var_value])
        return -cvar_estimation

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for asset in range(n_assets))
    initial_weights = np.ones(n_assets) / n_assets

    result = minimize(calculate_cvar, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    optimal_weights = result.x
    cvar_value = calculate_cvar(optimal_weights)

    return optimal_weights, -cvar_value

In [31]:
# Uso del ejemplo
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA','BTC-USD','^GSPC','^DJI','^IXIC','GC=F']
start_date = '2022-01-01'
end_date = '2023-01-01'

optimal_weights, portfolio_cvar = optimize_portfolio_kde_cvar(tickers, start_date, end_date, risk_profile='medio')

print("Pesos óptimos:", optimal_weights)
print(f"CVaR del portafolio: {portfolio_cvar:.4f}")

[*********************100%***********************]  10 of 10 completed

Pesos óptimos: [2.14616289e-17 6.26917383e-18 5.68660256e-03 7.18654570e-01
 8.38799786e-18 3.15024491e-19 3.02759962e-17 2.42636811e-01
 3.30220155e-02 2.80947559e-19]
CVaR del portafolio: -0.0140


In [32]:
inversion = 1000  # dólares
cvar_dinero = portfolio_cvar * inversion
print(f"Pérdida esperada en el peor 5% de los casos: ${cvar_dinero:.2f}")

inversion_por_activo = optimal_weights * inversion
for ticker, monto in zip(tickers, inversion_por_activo):
    print(f"Invertir ${monto:.2f} en {ticker}")

Pérdida esperada en el peor 5% de los casos: $-14.03
Invertir $0.00 en AAPL
Invertir $0.00 en MSFT
Invertir $5.69 en GOOGL
Invertir $718.65 en AMZN
Invertir $0.00 en TSLA
Invertir $0.00 en BTC-USD
Invertir $0.00 en ^GSPC
Invertir $242.64 en ^DJI
Invertir $33.02 en ^IXIC
Invertir $0.00 en GC=F
